In [ ]:
import numpy as np
import math
import itertools
import pandas as pd
import random
import matplotlib.pyplot as plt

In [ ]:
prices = {'A': 2500, 'B': 900, 'C': 830, 'D': 820, 'E': 1000}
preferences_god_view = {'A': 1, 'B': 1.5, 'C': 2, 'D': 2.5, 'E': 3}
user_preferences = {'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1}
all_combinations = np.array(list(itertools.combinations(preferences_god_view, 2)))
expect = []
expect_prob = []

In [ ]:
all_combinations

In [ ]:
all_combinations[0]
len(all_combinations[0])

In [ ]:
def BTL(S_t, user_preferences):
    # Calculate probabilities using Bradley-Terry-Luce model
    exp_preference_program=[]
    total_exp = 1 #include no purchase

    for i in range(len(S_t)):
      temp = np.exp(user_preferences[S_t[i]])
      total_exp = total_exp + temp
      exp_preference_program.append(temp)

    exp_no_purchase = 1
    p_item = [1/total_exp]
    for j in range(len(S_t)):
      temp = exp_preference_program[j] / total_exp
      #print(exp_preference_program[j])
      p_item.append(temp)
    return p_item

def calculate_single_expectation(current_S_t, programs_prices, user_preferences):
  purchase_probability_vector_by_mu = BTL(current_S_t, user_preferences)
  coresponding_prices=[0]
  for i in range(len(purchase_probability_vector_by_mu)-1):
    temp = programs_prices[current_S_t[i]]
    coresponding_prices.append(temp)
  #print(purchase_probability_vector_by_mu)
  #print(coresponding_prices)

  expectation = np.sum(np.array(purchase_probability_vector_by_mu) * np.array(coresponding_prices))

  return purchase_probability_vector_by_mu, expectation

In [ ]:
BTL(['A', 'B', 'C'], preferences_god_view) # for example

In [ ]:
calculate_single_expectation(['A', 'B'], prices, preferences_god_view) # for example

In [ ]:
def epilson_greedy_mab(epilson, S_t, programs_combinations):
  # 設定 S_t 出現的機率為 1 - epilson，其他的機率平均分配epilson
  probabilities = [1 - epilson if strategy == S_t else (epilson) / (len(programs_combinations) - 1) for strategy in programs_combinations]
  # 根據機率選擇策略
  selected_S_t = random.choices(programs_combinations, probabilities)[0]
  return selected_S_t
def find_best_expectation(programs_prices, user_preferences, how_many_a_set_has):
  programs_combinations = list(itertools.combinations(user_preferences, 2))
  print("programs_combinations:", programs_combinations)

  purchase_probability_vector_by_mu_list = []
  expectation_list = []
  for current_S_t in programs_combinations:
    purchase_probability_vector_by_mu, expectation = calculate_single_expectation(current_S_t, programs_prices, user_preferences)
    purchase_probability_vector_by_mu_list.append(purchase_probability_vector_by_mu)
    expectation_list.append(expectation)

  data = {
    'S_t': programs_combinations,
    'Purchase Probability': purchase_probability_vector_by_mu_list,
    'Expectation': expectation_list
  }
  df = pd.DataFrame(data)
  # df = df.round({'Purchase Probability': 2, 'Expectation': 2})
  print(df)

  best_S_t = programs_combinations[expectation_list.index(max(expectation_list))]
  best_expectation = expectation_list[expectation_list.index(max(expectation_list))]

  selected_S_t = epilson_greedy_mab(0.5, best_S_t, programs_combinations)
  selected_expectation = expectation_list[programs_combinations.index(selected_S_t)]

  print("best_S_t:", best_S_t, "expectation = ", best_expectation)
  print("selected_S_t:", selected_S_t, "expectation = ", selected_expectation)
  if best_S_t == selected_S_t:
    print("-------------Exploitation-------------")
  else:
    print("-------------Exploration--------------")
  return selected_S_t

In [ ]:
find_best_expectation(prices, preferences_god_view, 2) #from God's view

In [ ]:
selected_S_t=['A', 'B']
temp_user_preference={}
for strategy in selected_S_t:
    temp_user_preference[strategy] = 0
print(temp_user_preference)

In [ ]:
def generate_U_by_Gumbel(user_preferences):
  #Gumbel 分佈的均值是歐拉常數γ，約為 0.57721
  random_vars = np.random.gumbel(size=len(user_preferences), scale = 0.01)
  randomized_user_preference = {item: max(0, pref + rv) for (item, pref), rv in zip(user_preferences.items(), random_vars)}
  #print("U(randomized_user_preference)this iteration", randomized_user_preference)
  return randomized_user_preference

def user_purchase_decision_generator(selected_S_t, randomized_user_preference):
  purchase_probability_vector_by_U = BTL(selected_S_t, randomized_user_preference)
  # print("purchase prob: ", purchase_probability_vector_by_U)

  purchase_decision_index = np.random.choice(len(purchase_probability_vector_by_U), p=purchase_probability_vector_by_U)

  if purchase_decision_index == 0:
    print("user_purchase_decision", "buy nothing")
    return 0
  else:
    print("user_purchase_decision", "buy", selected_S_t[purchase_decision_index-1])
    return purchase_decision_index


def user_purchase_histories_updater(selected_S_t, user_purchase_decision, user_purchase_histories):
  if user_purchase_decision == 0:
    user_purchase_histories[selected_S_t][0]+=1
  else:
    user_purchase_histories[selected_S_t][user_purchase_decision]+=1

  return user_purchase_histories

def user_preference_updater(selected_S_t, user_preferences, user_purchase_histories):
  if np.any(user_purchase_histories[selected_S_t] == 0):
    return user_preferences

  total_decision_by_selected_S_t = 0
  for i in range(len(user_purchase_histories[selected_S_t])):
    total_decision_by_selected_S_t = user_purchase_histories[selected_S_t][i]
  user_purchase_probabilities_by_purchase_history = []
  for i in range(len(user_purchase_histories[selected_S_t])):
    user_purchase_probabilities_by_purchase_history.append(user_purchase_histories[selected_S_t][i] / total_decision_by_selected_S_t)

  temp_user_preference = {}
  for strategy in selected_S_t:
    temp_user_preference[strategy] = 0
  i = 1
  for strategy in selected_S_t:
    temp_user_preference[strategy] = math.log(user_purchase_probabilities_by_purchase_history[i] / user_purchase_probabilities_by_purchase_history[0])
    i = i + 1

  print("temp_user_preference", temp_user_preference)
  print("user_preferences", user_preferences)

  for j in range(len(selected_S_t)):
    user_preferences[selected_S_t[j]] = (user_preferences[selected_S_t[j]])*0.2 + (temp_user_preference[selected_S_t[j]])*0.8
  print("updated_user_preferences", user_preferences)
  print("--------------------------------------")
  return user_preferences

In [ ]:
def normalized(x, listOf):
  x_normalized = (x - min(listOf)) / (max(listOf) - min(listOf))
  return x_normalized


In [ ]:
programs_prices = {'A': 2500, 'B': 900, 'C': 830, 'D': 820, 'E': 1000}
#user_preferences_god_view = [{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}, {'A': 2, 'B': 2, 'C': 3, 'D': 4, 'E': 3}]
user_preferences_god_view = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
all_combinations = np.array(list(itertools.combinations(preferences_god_view, 2)))
expect = []
expect_prob = []

# 10 means 旅行商所有推薦組合, 3 ,means推薦某個組合時會有的 3 個情況
user_purchase_histories = {}

# 創建所有可能的從 'A' 到 'E' 的排列組合
locations = ['A', 'B', 'C', 'D', 'E']
combinations = list(itertools.combinations(locations, 2))
# 對每個字典進行初始化
for combination in combinations:
  user_purchase_histories[combination] = np.array([0, 0, 0])


--------------------------------------------------------------------------------

In [ ]:
def thompson_sampling_mab(programs_combinations, successes, failures):
  # 使用Beta分佈來表示每個策略的概率
  beta_parameters = [(successes[strategy] + 1, failures[strategy] + 1) for strategy in programs_combinations]

  # 從每個策略的Beta分佈中抽樣一個值，並選擇具有最高抽樣值的策略
  sampled_values = [np.random.beta(alpha, beta) for alpha, beta in beta_parameters]
  print(sampled_values)

  #combined_scores = [sampled_value + price for sampled_value, price in zip(sampled_values, programs_prices)]

  selected_index = np.argmax(sampled_values)
  selected_S_t = programs_combinations[selected_index]

  return selected_S_t

In [ ]:
programs_combinations = list(itertools.combinations(user_preferences, 2))
successes = {}
failures = {}
for strategy_pair in programs_combinations:
  successes[strategy_pair] = 0
  failures[strategy_pair] = 0
thompson_sampling_mab(programs_combinations, successes, failures)

In [ ]:
def find_best_expectation_thompson(programs_prices, user_preferences, successes, failures, how_many_a_set_has):
  programs_combinations = list(itertools.combinations(user_preferences, how_many_a_set_has))
  print("programs_combinations:", programs_combinations)

  purchase_probability_vector_by_mu_list = []
  expectation_list = []
  for current_S_t in programs_combinations:
    purchase_probability_vector_by_mu, expectation = calculate_single_expectation(current_S_t, programs_prices, user_preferences)
    purchase_probability_vector_by_mu_list.append(purchase_probability_vector_by_mu)
    expectation_list.append(expectation)

  data = {
    'S_t': programs_combinations,
    'Purchase Probability': purchase_probability_vector_by_mu_list,
    'Expectation': expectation_list
  }
  df = pd.DataFrame(data)
  # df = df.round({'Purchase Probability': 2, 'Expectation': 2})
  # print(df)

  best_S_t = programs_combinations[expectation_list.index(max(expectation_list))]
  best_expectation = expectation_list[expectation_list.index(max(expectation_list))]

  # selected_S_t = epilson_greedy_mab(0.5, best_S_t, programs_combinations)
  # selected_expectation = expectation_list[programs_combinations.index(selected_S_t)]

  selected_S_t = thompson_sampling_mab(programs_combinations, successes, failures)
  selected_expectation = expectation_list[programs_combinations.index(selected_S_t)]

  # print("best_S_t:", best_S_t, "expectation = ", best_expectation)
  # print("selected_S_t:", selected_S_t, "expectation = ", selected_expectation)
  if best_S_t == selected_S_t:
    print("-------------Exploitation-------------")
  else:
    print("-------------Exploration--------------")
  return selected_S_t

In [ ]:
iterations = 1000
times = 500
how_many_a_set_has = 2
final = []

for j in range(times):
    all_earn = 0
    a=[]
    b=[]
    user_purchase_decision = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
    cumulation = []
    select=[]
    successes = {}
    failures = {}
    for strategy_pair in programs_combinations:
        successes[strategy_pair] = 0
        failures[strategy_pair] = 0

    for combination in combinations:
        user_purchase_histories[combination] = np.array([0, 0, 0])

    for i in range(iterations):
        print("iteration", i)
        selected_S_t = find_best_expectation_thompson(programs_prices, user_preferences, successes, failures, how_many_a_set_has)
        randomized_user_preference = generate_U_by_Gumbel(user_preferences_god_view)
        user_purchase_decision = user_purchase_decision_generator(selected_S_t, randomized_user_preference)
        user_purchase_histories = user_purchase_histories_updater(selected_S_t, user_purchase_decision, user_purchase_histories)
        p = programs_prices[selected_S_t[user_purchase_decision-1]]

        select.append(selected_S_t)

        if user_purchase_decision != 0 and p > 1200:
            successes[selected_S_t] += 1
        else:
            failures[selected_S_t] += 1

        a.append(user_preferences['E'])

        all_earn = all_earn + programs_prices[selected_S_t[user_purchase_decision-1]]
        cumulation.append(all_earn)

        user_preferences = user_preference_updater(selected_S_t, user_preferences, user_purchase_histories)
        # print("mu(user_preferences):", user_preferences)
        # print("history:　", user_purchase_histories)
        # print("______________________________________________________________________________________________")

        b.append(user_preferences['E'])
    final.append(all_earn)

print(final)

In [ ]:
import matplotlib.pyplot as plt

# 定義組合
combinations = [('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'E'), ('D', 'E')]

# 迭代次數
iterations = 1000

# 以下資料為散布圖所用的1000筆資料
selected_combinations = [('D', 'E'), ('B', 'D'), ('B', 'C'), ('C', 'D'), ('B', 'E'), ('A', 'C'), ('C', 'E'), ('B', 'C'), ('A', 'D'), ('C', 'E'), ('A', 'E'), ('B', 'E'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('C', 'D'), ('A', 'B'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('B', 'C'), ('A', 'E'), ('C', 'E'), ('A', 'B'), ('C', 'D'), ('D', 'E'), ('C', 'D'), ('B', 'D'), ('A', 'C'), ('A', 'B'), ('B', 'D'), ('A', 'E'), ('A', 'B'), ('B', 'C'), ('C', 'E'), ('B', 'E'), ('A', 'B'), ('B', 'E'), ('D', 'E'), ('B', 'D'), ('B', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('B', 'E'), ('A', 'D'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'B'), ('A', 'E'), ('A', 'C'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('B', 'D'), ('D', 'E'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('B', 'C'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('A', 'E'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('B', 'D'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('B', 'E'), ('A', 'B'), ('C', 'D'), ('C', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'C'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('B', 'E'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('B', 'E'), ('A', 'E'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('C', 'E'), ('A', 'E'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('C', 'E'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('C', 'E'), ('A', 'E'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('A', 'B'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'E'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('D', 'E'), ('A', 'B'), ('A', 'B'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('B', 'D'), ('A', 'B'), ('A', 'B'), ('B', 'E'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'E'), ('C', 'E'), ('A', 'B'), ('B', 'C'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('C', 'D'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B')]

# 將組合轉换成字符串形式，以便绘制
combinations_str = [f'{c[0]} - {c[1]}' for c in combinations]

# 將選定的組合轉换成相應的索引以便绘制
selected_combination_indices = [combinations.index(c) for c in selected_combinations]

# 创建一个散布图
plt.figure(figsize=(10, 6))
plt.scatter(range(iterations), selected_combination_indices, marker='.', alpha=0.5)

# 设置標籤
plt.xlabel('Iterations')
plt.ylabel('Selected Combinations')
plt.yticks(range(len(combinations)), combinations_str)
plt.title('Scatter Plot of Selected Combinations Over Iterations')

# 顯示圖形
plt.show()


In [ ]:
time_points = list(range(0, (2500 +1), 100))
plt.plot(time_points, compare_earn, marker='o')
# 添加標題和標籤
plt.title('Compare threshold')
plt.xlabel('Threshold-the definition of prices')
plt.ylabel('earn')

# 顯示圖形
plt.show()

In [ ]:
values = list(user_preferences.values())
normalized_values = [value/sum(values) for value in values]
normalized_user_preference = {key: normalized_value for key, normalized_value in zip(user_preferences.keys(), normalized_values)}
normalized_user_preference

In [ ]:
time_points = list(range(1, (times * iterations + 1)))
plt.plot(time_points, a, marker='o')
# 添加標題和標籤
plt.title('Update Trend Graph')
plt.xlabel('Time')
plt.ylabel('Preference Update of E')

# 顯示圖形
plt.show()